In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import sys
project_path = os.path.abspath(os.path.join('..'))

if project_path not in sys.path:
    sys.path.append(project_path)
    

In [17]:
from src.features.generate_features import *
from src.preprocess.preprocess_ATSF import process
## change path
# from model_pipeline import *

In [18]:
data = pd.read_csv('../data/kaggle/data_kaggle.csv')
data = process(data)

In [11]:
config_filename = '../src/configs/gen_features_config.json'
model_config_filename = '../src/configs/model_config.json'

In [19]:
features_json = read_json(config_filename)
data[features_json['date_col']] = pd.to_datetime(data[features_json['date_col']], dayfirst=True)

In [20]:
data_features = generate_lagged_features(data=data,
                         target_cols=features_json['target_cols'],
                         id_cols=features_json['id_cols'],
                         date_col=features_json['date_col'],
                         lags=features_json['lags'],
                         windows=features_json['windows'],
                         preagg_methods=features_json['preagg_methods'],
                         agg_methods=features_json['agg_methods'],
                         dynamic_filters=features_json['dynamic_filters'],
                         ewm_params=features_json['ewm_params']
                        )

IntProgress(value=0, max=1)

In [14]:
choose_model(model_config_filename)

NameError: name 'choose_model' is not defined

In [8]:
model_config = read_json(model_config_filename)
res_test = pipeline(data_features, model_config)

Preprocessing:


  0%|          | 0/2420 [00:00<?, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 1.4 K 
3  | prescalers                         | ModuleDict                      | 144   
4  | static_variable_selection          | VariableSelectionNetwork        | 1.5 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 3.4 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 3.4 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 460   
8  | static_

Training: 0it [00:00, ?it/s]

Biulding secondary model:
Preprocessing:


...

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0
---------


  0%|          | 0/340 [00:00<?, ?it/s]

  0%|          | 0/340 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [9]:
res_test

,SKU_id,Store_id,Date,Demand
0,1,1,2016-05-23,156.448517
1,1,1,2016-05-24,45.283524
2,1,1,2016-05-25,52.282661
3,1,1,2016-05-26,50.207172
4,1,1,2016-05-27,52.015919
...,...,...,...,...
5965,2,110,2016-06-15,11.916748
5966,2,110,2016-06-16,7.261536
5967,2,110,2016-06-17,43.00148
5968,2,110,2016-06-18,19.580734


In [10]:
res_test[model_config['target']].to_csv('ans.csv', index_label='id')